In [1]:
import json
import inspect
import keras
import re
from importlib import import_module
from copy import deepcopy
import pkg_resources


Using TensorFlow backend.


In [2]:
toolbox = 'Keras'
excluded_classes = ['K']
layers = [layer for layer in dir(keras.layers) 
          if layer[0].isupper() and layer not in excluded_classes]

In [3]:
def keras2giraffe(node, section):
    try:
        inputs = node().get_config()
    except:
        return {}
    
    ports = [{ 'name': port,
               'input': False,
               'output': False,
               'visible': True,
               'editable': True,
               'default': inputs[port], 
               'code': []
             } for port in inputs.keys()];
    ports.insert(0, {
        'name': 'INPUT',
        'input': True,
        'output': False,
        'visible': True,
        'editable': True, 
        'code': []
    })
    ports.append({
        'name': 'OUTPUT',
        'input': False,
        'output': True,
        'visible': True,
        'editable': True,
        'code': []
    })
    
    return { 
        'name': name,
        'category': [section],
        'toolbox': toolbox,
        'code': [{
            'import': 'import keras.layers.%s' % name,
        }],
        'web_url': 'https://keras.io/layers/%s/%s' % (section, name),
        'ports': ports
    }

In [4]:
pattern = re.compile("from .(?P<name>[a-z]+) import *")
keras_layers_init = pkg_resources.resource_string('keras.layers', '__init__.py')
lines = [line.decode("utf-8") for line in keras_layers_init.split(b'\n') if pattern.match(line.decode("utf-8"))]


categories = []
for line in lines:
    module_name = pattern.match(line).group("name")
    module = import_module('keras.layers.%s' % module_name)
    node_list = [getattr(module, n) for n in dir(module) if n[0].isupper() or n[0].isdigit()]
    
    nodes = [keras2giraffe(node, module_name) for node in node_list]
    nodes = [node for node in nodes if bool(node)]
    categories.append({ 'name': module_name, 'nodes': nodes })
   
dictionary = { 'name': toolbox, 'categories': categories }    

NameError: name 'name' is not defined

In [ ]:
with open('keras_nodes.json', 'w') as outfile:
    json.dump({'toolboxes': [dictionary]}, outfile, sort_keys=False, indent=2)
    # json.dump({'toolboxes': toolboxes}, outfile, separators=(',', ':'))